In [11]:
# All required imports for API Requeest of data base
import requests
import json
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

# All required imports for word extraction and analysis
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import string
import matplotlib.pyplot as plt

## 1. Get data from MongoDB Atlas Database

In [32]:
disable_warnings(InsecureRequestWarning)

url = "https://eu-central-1.aws.data.mongodb-api.com/app/data-lkdyd/endpoint/data/v1/action/find"
payload = json.dumps({
    "collection": "wartung-log-ausfall",
    "database": "data-project",
    "dataSource": "Cluster0"
})
headers = {
  'Content-Type': 'application/json',
  'Access-Control-Request-Headers': '*',
  'api-key': 'Mlu4wzPwd6o1I1WX7XEMsopBkSZgUx6qpbewbFpwxLHnz0lkhJY5AY47NpDYzZV0',
}
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
print(response.text)

{"documents":[{"_id":"65b95fd9e165317b9362ef79","MesswertID":12478,"SystemID":"A1","Datum":"01.02.2022","Zeit":"06:00","CPU":50,"RAM":50,"HD":20,"Network":30,"ServiceOK":1,"LogLevel":"Info","ServiceState":34},{"_id":"65b95fd9e165317b9362ef7a","MesswertID":12479,"SystemID":"A1","Datum":"01.02.2022","Zeit":"06:05","CPU":60,"RAM":60,"HD":20,"Network":30,"ServiceOK":1,"LogLevel":"Info","ServiceState":24},{"_id":"65b95fd9e165317b9362ef7b","MesswertID":12480,"SystemID":"A1","Datum":"01.02.2022","Zeit":"06:10","CPU":60,"RAM":60,"HD":20,"Network":30,"ServiceOK":0,"LogLevel":"Error","LogMessage":"Could not connect to database","ServiceState":99},{"_id":"65b95fd9e165317b9362ef7c","MesswertID":12481,"SystemID":"A1","Datum":"01.02.2022","Zeit":"06:15","CPU":20,"RAM":25,"HD":25,"Network":35,"ServiceOK":1,"LogLevel":"Info","ServiceState":7},{"_id":"65b95fd9e165317b9362ef7d","MesswertID":12482,"SystemID":"A1","Datum":"01.02.2022","Zeit":"06:20","CPU":35,"RAM":30,"HD":25,"Network":30,"ServiceOK":1,"Lo

In [55]:
# Convert the response to a json object
json_data = json.loads(response.text)
json_data

{'documents': [{'_id': '65b95fd9e165317b9362ef79',
   'MesswertID': 12478,
   'SystemID': 'A1',
   'Datum': '01.02.2022',
   'Zeit': '06:00',
   'CPU': 50,
   'RAM': 50,
   'HD': 20,
   'Network': 30,
   'ServiceOK': 1,
   'LogLevel': 'Info',
   'ServiceState': 34},
  {'_id': '65b95fd9e165317b9362ef7a',
   'MesswertID': 12479,
   'SystemID': 'A1',
   'Datum': '01.02.2022',
   'Zeit': '06:05',
   'CPU': 60,
   'RAM': 60,
   'HD': 20,
   'Network': 30,
   'ServiceOK': 1,
   'LogLevel': 'Info',
   'ServiceState': 24},
  {'_id': '65b95fd9e165317b9362ef7b',
   'MesswertID': 12480,
   'SystemID': 'A1',
   'Datum': '01.02.2022',
   'Zeit': '06:10',
   'CPU': 60,
   'RAM': 60,
   'HD': 20,
   'Network': 30,
   'ServiceOK': 0,
   'LogLevel': 'Error',
   'LogMessage': 'Could not connect to database',
   'ServiceState': 99},
  {'_id': '65b95fd9e165317b9362ef7c',
   'MesswertID': 12481,
   'SystemID': 'A1',
   'Datum': '01.02.2022',
   'Zeit': '06:15',
   'CPU': 20,
   'RAM': 25,
   'HD': 25,
   '

In [61]:
# Get response data as a dataframe
df = pd.DataFrame(json_data)
df

,documents
0,"{'_id': '65b95fd9e165317b9362ef79', 'MesswertI..."
1,"{'_id': '65b95fd9e165317b9362ef7a', 'MesswertI..."
2,"{'_id': '65b95fd9e165317b9362ef7b', 'MesswertI..."
3,"{'_id': '65b95fd9e165317b9362ef7c', 'MesswertI..."
4,"{'_id': '65b95fd9e165317b9362ef7d', 'MesswertI..."
...,...
995,"{'_id': '65b95fd9e165317b9362f35c', 'MesswertI..."
996,"{'_id': '65b95fd9e165317b9362f35d', 'MesswertI..."
997,"{'_id': '65b95fd9e165317b9362f35e', 'MesswertI..."
998,"{'_id': '65b95fd9e165317b9362f35f', 'MesswertI..."


Je nach Use Case hier noch JSON bearbeiten, bevor es in das DF überführt wird

In [81]:
# Lower all the text in the dataframe
import string

df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df['LowText']

C:\Users\betz\AppData\Local\Temp\ipykernel_16740\4045279393.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


0      {'_id': '65b95fd9e165317b9362ef79', 'messwerti...
1      {'_id': '65b95fd9e165317b9362ef7a', 'messwerti...
2      {'_id': '65b95fd9e165317b9362ef7b', 'messwerti...
3      {'_id': '65b95fd9e165317b9362ef7c', 'messwerti...
4      {'_id': '65b95fd9e165317b9362ef7d', 'messwerti...
                             ...                        
995    {'_id': '65b95fd9e165317b9362f35c', 'messwerti...
996    {'_id': '65b95fd9e165317b9362f35d', 'messwerti...
997    {'_id': '65b95fd9e165317b9362f35e', 'messwerti...
998    {'_id': '65b95fd9e165317b9362f35f', 'messwerti...
999    {'_id': '65b95fd9e165317b9362f360', 'messwerti...
Name: LowText, Length: 1000, dtype: object

## 2. Daten für Analyse vorbereiten
Verwendete Verfahren:
- Tokenization
- Lemmatization
- Stemming

In [82]:
log_file = df['LowText']
log_file

0      {'_id': '65b95fd9e165317b9362ef79', 'messwerti...
1      {'_id': '65b95fd9e165317b9362ef7a', 'messwerti...
2      {'_id': '65b95fd9e165317b9362ef7b', 'messwerti...
3      {'_id': '65b95fd9e165317b9362ef7c', 'messwerti...
4      {'_id': '65b95fd9e165317b9362ef7d', 'messwerti...
                             ...                        
995    {'_id': '65b95fd9e165317b9362f35c', 'messwerti...
996    {'_id': '65b95fd9e165317b9362f35d', 'messwerti...
997    {'_id': '65b95fd9e165317b9362f35e', 'messwerti...
998    {'_id': '65b95fd9e165317b9362f35f', 'messwerti...
999    {'_id': '65b95fd9e165317b9362f360', 'messwerti...
Name: LowText, Length: 1000, dtype: object

In [83]:
#Stemming
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer("english")
print("--->Stemming:")
log_file['Stem'] = [snowball.stem(word) for word in log_file]
print(log_file['Stem'])

--->Stemming:
["{'_id': '65b95fd9e165317b9362ef79', 'messwertid': 12478, 'systemid': 'a1', 'datum': '01.02.2022', 'zeit': '06:00', 'cpu': 50, 'ram': 50, 'hd': 20, 'network': 30, 'serviceok': 1, 'loglevel': 'info', 'servicestate': 34}", "{'_id': '65b95fd9e165317b9362ef7a', 'messwertid': 12479, 'systemid': 'a1', 'datum': '01.02.2022', 'zeit': '06:05', 'cpu': 60, 'ram': 60, 'hd': 20, 'network': 30, 'serviceok': 1, 'loglevel': 'info', 'servicestate': 24}", "{'_id': '65b95fd9e165317b9362ef7b', 'messwertid': 12480, 'systemid': 'a1', 'datum': '01.02.2022', 'zeit': '06:10', 'cpu': 60, 'ram': 60, 'hd': 20, 'network': 30, 'serviceok': 0, 'loglevel': 'error', 'logmessage': 'could not connect to database', 'servicestate': 99}", "{'_id': '65b95fd9e165317b9362ef7c', 'messwertid': 12481, 'systemid': 'a1', 'datum': '01.02.2022', 'zeit': '06:15', 'cpu': 20, 'ram': 25, 'hd': 25, 'network': 35, 'serviceok': 1, 'loglevel': 'info', 'servicestate': 7}", "{'_id': '65b95fd9e165317b9362ef7d', 'messwertid': 124

In [87]:
# Tokenization
from nltk.tokenize import word_tokenize
print("--->Tokenization:")
log_file['Token'] = [word_tokenize(word) for word in log_file]
print(log_file['Token'])

--->Tokenization:


TypeError: expected string or bytes-like object, got 'list'

In [79]:
# Lemmatization in english language
from nltk.stem import WordNetLemmatizer
print("--->Lemmatization:")
log_file['Lem'] = [' '.join([WordNetLemmatizer(wd) for wd in word]) for word in log_file['Token']]
print(log_file['Lem'])

--->Lemmatization:


KeyError: 'Token'

## 3. Analyse durchführen